In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import numpy as np
from tqdm import trange

#
from visualize import Visualize
from track import Track, detect_id_switch3

#
import cv2
import os

#
from tqdm import tqdm

Autosaving every 180 seconds


In [2]:

def extract_movie_frame(fname_movie,
                        frame_number):
    
    import cv2
    cap = cv2.VideoCapture(fname_movie)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    res, frame0 = cap.read()
    res, frame1 = cap.read()

    return frame0, frame1

########################################
########################################
########################################
def detect_id_switch(fname_movie,
                     tracks, 
                     animal_id,
                     min_n_features=4):
    
    #
    other_animal_ids = np.int32(np.delete(np.arange(6), animal_id))
   # print (other_animal_ids)
    
    
    # check for continuity in a track
    track = tracks[:,animal_id]
    #print (track.shape)
    
    # calculate the feature-wise distance between current frame and next frame
    ctr=0
    for k in trange(tracks.shape[0]-1):
        feats0 = track[k]
        
        for a in other_animal_ids:
            feats1 = tracks[k+1,a]
            
            #
            dists = np.linalg.norm(feats1-feats0,axis=1)
            #print (dists)

            # 
            sums = np.nanmedian(dists)
            #print (sums)
            
            if sums<10:
                n = np.count_nonzero(np.isnan(dists))
                if n<=(6-min_n_features):
                    #print (n)
                    #print (dists)
                    #print (sums)
                    frame0, frame1 = extract_movie_frame(fname_movie,
                                                k)
                    #if ctr<5:
                    plot_flip2(k,
                                   frame0,
                                   frame1,
                                   tracks[k],
                                   tracks[k+1]
                                  )

                    ctr+=1
                #break
                
        #if sums<10:
        #    break


########################################
########################################
########################################

def plot_flip2(n,
               frame0,
               frame1,
               track0,
              track1):
    
    clrs = ['blue','red','pink','cyan','magenta','lightgreen',
    ]
    
    
    #print ("track0: ", track0)
    
    plt.figure(figsize=(15,15))
    
    ax=plt.subplot(121)
    ax.imshow(frame0)
    
    for k in range(6):
        feats0 = track0[k]
        #print("Feats0: ", feats0, feats0.shape)
        if len(feats0.shape)>1:
    
            plt.scatter(feats0[:,0],
                   feats0[:,1],
                   c=clrs[k])
    
    #
    ax=plt.subplot(122)
    ax.imshow(frame1)
    for k in range(6):
        feats1 = track1[k]
        #print("Feats1: ", feats1)
        
        if len(feats1.shape)>1:
            plt.scatter(feats1[:,0],
                    feats1[:,1],
                    c=clrs[k])
    
    plt.savefig('/home/cat/example_'+str(n)+'.png')
    plt.close()
    
########################################
########################################
########################################

def plot_flip(frame0,
              frame1,
              animal_id,
              feats0,
              feats1,
             a):
    
    clrs = ['blue','red','pink','cyan','magenta','lightgreen',
    
    
    ]
    
    plt.figure()
    
    ax=plt.subplot(121)
    ax.imshow(frame0)
    plt.scatter(feats0[:,0],
               feats0[:,1],
               c=clrs[animal_id])
    
    #
    ax=plt.subplot(122)
    ax.imshow(frame1)
    plt.scatter(feats1[:,0],
               feats1[:,1],
               c=clrs[a])
    

    plt.show()
    

In [3]:
######################################################## 
################ GENERATE FIXED TRACK ##################
########################################################

fnames = np.loadtxt('/media/cat/256GB/dan/id_switch/fnames.txt',dtype='str')

for fname_slp in fnames:
    # 
    #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
    track = Track(fname_slp)
    track.track_type = 'features'
    track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
    track.load_tracks()

    print (track.tracks.shape)


track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_11_06_03_274369_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_14_11_50_567584_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), type "<f8">
(28802, 6, 6, 2)
track_names: 6
node_names: 6
tracks: (28802, 6, 2, 6)
track_occupancy: (6, 28802)
Saved as /media/cat/256GB/dan/id_switch/day/2020_07_30_17_34_43_625922_compressed_Day.h5
group2:  <HDF5 dataset "tracks": shape (6, 2, 6, 28802), t

In [7]:

def plot_feats(feats,
           colors,
           img_out,
           dot_size):

    for a in range(feats.shape[0]):
        color = colors[a]  # (255, 0, 0)
        #print ("animal id: ", a, ", color: ", color)
        for f in range(feats.shape[1]):
            x = feats[a, f, 0]
            y = feats[a, f, 1]

            if np.isnan(x) or np.isnan(y):
                continue

            x = int(x)
            y = int(y)

            # plot cicrl
            center_coordinates = (x, y)
            img_out = cv2.circle(img_out, 
                                 center_coordinates, 
                                 dot_size, 
                                 color, 
                                 -1)
   # print ('')
    return img_out

#
def detect_id_switch_no_plot(fname_movie,
                             tracks, 
                             animal_id,
                             min_n_features=4):

    #
    other_animal_ids = np.int32(np.delete(np.arange(6), animal_id))
    
    # select animal_id track
    track = tracks[:,animal_id]
    
    # calculate the feature-wise distance between current frame and next frame
    ctr=0
    frame_ids = []
    corner=300
    for k in trange(tracks.shape[0]-1, desc='finding id switches in movie'):
        feats0 = track[k]
        
        for a in other_animal_ids:
            feats1 = tracks[k+1,a]
            
            #
            dists = np.linalg.norm(feats1-feats0,axis=1)

            # 
            med = np.nanmedian(dists)
            #n = np.count_nonzero(np.isnan(dists))
            # if (6-n)>0:
            #     print ("n: ", n, ", median distances: ", med, ", dists: ", dists)
            
            #
            if med<15:
                #
                n = np.count_nonzero(np.isnan(dists))
                #if n<=(6-min_n_features):
                if n==0:
                    # also skip switches in the huddle
                    if np.nanmedian(feats0[:,0])>corner:
                        if np.nanmedian(feats0[:,1])>corner:

                            frame_ids.append([k,k+1])

                        ctr+=1
                #else:
                #    print ("n toolow: ", n)
    print ("found: ", len(frame_ids), " switches")
    return frame_ids


In [3]:

#
animal_ids = np.arange(6)
all_frames = []
    
fname_slp = '/media/cat/256GB/dan/id_switch/day/2020_07_30_17_34_43_625922_compressed_Day.slp'
fname_slp = '/media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
track = Track(fname_slp)
track.track_type = 'features'
track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
track.load_tracks()


#### FIND ALL ID SWITCHES IN MOVIE ####
all_frames = []

for animal_id in animal_ids:
#for animal_id in [2]:
    frame_ids = detect_id_switch3(track.tracks, 
                                  animal_id)
    if len(frame_ids)>0:
        all_frames.append(frame_ids)


finding id switches in movie: 100%|██████████| 28801/28801 [00:08<00:00, 3297.92it/s]


found:  0  switches


finding id switches in movie: 100%|██████████| 28801/28801 [00:06<00:00, 4183.17it/s]


found:  0  switches


finding id switches in movie: 100%|██████████| 28801/28801 [00:07<00:00, 4073.96it/s]


...

found:  0  switches


finding id switches in movie: 100%|██████████| 28801/28801 [00:07<00:00, 4039.17it/s]


found:  3  switches


finding id switches in movie: 100%|██████████| 28801/28801 [00:08<00:00, 3506.40it/s]

found:  1  switches


In [4]:

all_frames = np.vstack(all_frames)
#print ("all frames: ", all_frames.shape)

idx = np.argsort(all_frames[:,0])
all_frames = all_frames[idx]

#
print (all_frames)


[[12959 12960     2     0]
 [13016 13017     2     0]
 [14522 14523     4     2]
 [14566 14567     2     4]
 [14658 14659     4     2]
 [14678 14679     2     4]
 [15137 15138     5     4]
 [15173 15174     4     5]
 [16416 16417     2     3]
 [16580 16581     2     3]]


In [8]:
#
import glob

#
colors = [
    (0, 0, 255),
    (255, 0, 0),
    (255, 255, 0),
    (0, 128, 0),
    (128,128, 128),
    (128, 0, 128),
]

#
animal_ids = np.arange(6)

# load and videos
#fname_out_annotated = fname_slp[:-4]+"_annotated.mp4"

# make new video video settings
size_vid = np.int32(np.array([900, 700]))
fps_out=1
dot_size = 4
thickness = -1
window = 100
# fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# #fourcc = cv2.VideoWriter_fourcc(*'X264')
# #fourcc = cv2.VideoWriter_fourcc(*args["codec"])
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#fourcc = cv2.VideoWriter_fourcc('p', 'n', 'g', '')

# load videos
fname_out = fname_slp[:-4]+'_daytime_hybrid.mp4' 
fname_out_cropped = fname_out[:-4]+"_cropped.mp4"
fname_out_annotated = fname_out[:-4]+"_annotated.mp4"
fname_out_annotated_cropped = fname_out[:-4]+"_annotated_cropped.mp4"

print ("Fname out: ", fname_out)

# make new video video settings
size_vid = np.int32(np.array([900, 700]))
fps_out=1
dot_size = int(6)
thickness = -1
video_out = cv2.VideoWriter(fname_out, 
                            fourcc, 
                            fps_out, 
                            (size_vid[0], size_vid[1]), 
                            True)

video_out_cropped = cv2.VideoWriter(fname_out_cropped, 
                            fourcc, 
                            fps_out, 
                            (size_vid[0], size_vid[1]), 
                            True)

#
video_out_annotated = cv2.VideoWriter(fname_out_annotated, 
                                      fourcc, 
                                      fps_out, 
                                      (size_vid[0], size_vid[1]), 
                                      True)

#
video_out_annotated_cropped = cv2.VideoWriter(fname_out_annotated_cropped, 
                                      fourcc, 
                                      fps_out, 
                                      (size_vid[0], size_vid[1]), 
                                      True)

############################################
track = Track(fname_slp)
track.track_type = 'features'
track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
track.load_tracks()

#
fname_movie = '/media/cat/256GB/dan/id_switch/day/2020_07_30_17_34_43_625922_compressed_defished_shrink_cropped.mp4'
fname_movie = '/media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4'
print ("movie: ", fname_movie)


# load current vid
original_vid = cv2.VideoCapture(fname_movie)
width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )

# set frames to new ones
ctr=0
for frames in tqdm(all_frames, desc='adding frames to movies'):
    #print ("frame #s: ", frames)
    if len(frames)==0:
        continue

    #
    animal_id = frames[2]

    #
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, frames[0])
    
    #
    for z in range(2):

        # find centre of animal black out surrounding area
        feats = track.tracks[frames[0]+z]

        #
        if z==0:
            #print ("feats: ", feats)
            #print ("feats: animal id: ", feats[animal_id])
            feats_centre = np.nanmedian(feats[animal_id],0)
            #print ("feats centre: ", feats_centre)
                            #
            min_x = int(max(feats_centre[0]-window,0))
            max_x = int(min(feats_centre[0]+window,900))
            min_y = int(max(feats_centre[1]-window,0))
            max_y = int(min(feats_centre[1]+window,700))


        ret, img_out = original_vid.read()

        ###################################      
        ########## REGULAR VIDS ###########
        ###################################  
        # save non-annotated video
        video_out.write(img_out)

        # crop the image
        img_out_cropped = img_out.copy()
        img_out_cropped[:,:min_x] = 0
        img_out_cropped[:,max_x:] = 0
        img_out_cropped[:min_y] = 0
        img_out_cropped[max_y:] = 0

        video_out_cropped.write(img_out_cropped)

        ###################################      
        ######### ANNOTATED VIDS ##########      
        ###################################      
        #

        #
        img_out = plot_feats(
                            feats,
                            colors,
                            img_out,
                            dot_size)
        #
        if True:
            #
            video_out_annotated.write(img_out)

            img_out_cropped = img_out.copy()
            img_out_cropped[:,:min_x] = 0
            img_out_cropped[:,max_x:] = 0
            img_out_cropped[:min_y] = 0
            img_out_cropped[max_y:] = 0            

            #
            video_out_annotated_cropped.write(img_out_cropped)


    #video_out_annotated.write(img_out*0)

#break
    
video_out.release()
video_out_cropped.release()
video_out_annotated.release()
video_out_annotated_cropped.release()
print ("DONE...")



Fname out:  /media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_Day_daytime_hybrid.mp4
movie:  /media/cat/256GB/dan/id_switch/day/2020_07_30_07_58_21_812233_compressed_defished_shrink_cropped.mp4


adding frames to movies: 100%|██████████| 10/10 [00:01<00:00,  6.98it/s]

DONE...


In [ ]:
/media/cat/256GB/dan/id_switch/day/vids/daytime_hybrid.mp4
/media/cat/256GB/dan/id_switch/day/vids


In [ ]:
# #
# def plot_feats(feats,
#            colors,
#            img_out,
#            dot_size):

#     for a in range(feats.shape[0]):
#         color = colors[a]  # (255, 0, 0)
#         #print ("animal id: ", a, ", color: ", color)
#         for f in range(feats.shape[1]):
#             x = feats[a, f, 0]
#             y = feats[a, f, 1]

#             if np.isnan(x) or np.isnan(y):
#                 continue

#             x = int(x)
#             y = int(y)

#             # plot cicrl
#             center_coordinates = (x, y)
#             img_out = cv2.circle(img_out, 
#                                  center_coordinates, 
#                                  dot_size, 
#                                  color, 
#                                  -1)
#    # print ('')
#     return img_out


# import glob
# fnames_list = '/media/cat/4TB/dan/id_switch/day/fnames.txt'
# fnames = np.loadtxt(fnames_list,dtype='str')


# #
# colors = [
#     (0, 0, 255),
#     (255, 0, 0),
#     (255, 255, 0),
#     (0, 128, 0),
#     (128,128, 128),
#     (128, 0, 128),
# ]


# #
# animal_ids = np.arange(6)

# # load and videos

# fname_out = os.path.split(fnames_list)[0]+'/movie_hybrid.mp4' 
# fname_out_annotated = fname_out[:-4]+"_annotated.mp4"

# # make new video video settings
# size_vid = np.int32(np.array([900, 700]))
# fps_out=1
# dot_size = int(6)
# thickness = -1
# fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# #fourcc = cv2.VideoWriter_fourcc(*'X264')
# #fourcc = cv2.VideoWriter_fourcc(*args["codec"])
# #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
# #
# video_out = cv2.VideoWriter(fname_out, 
#                            fourcc, 
#                            fps_out, 
#                            (size_vid[0], size_vid[1]), 
#                            True)

# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# fourcc = cv2.VideoWriter_fourcc(*'DIVX')

# #
# video_out = cv2.VideoWriter(fname_out,
#                             fourcc, 
#                             20.0, (900,700))


# print ("Saving video: ", fname_out)
# #
# video_out_annotated = cv2.VideoWriter(fname_out_annotated, 
#                                       fourcc, 
#                                       fps_out, 
#                                       (size_vid[0], size_vid[1]), 
#                                       True)
# # #
# for fname_slp in fnames:
#     print ("Loading: ", fname_slp)
    
#     #fname_slp = '/home/cat/data/dan/2020_07_30_00_34_28_393297/day/2020_07_30_07_58_21_812233_compressed_Day.slp'
#     track = Track(fname_slp)
#     track.track_type = 'features'
#     track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
#     track.load_tracks()
    
#     #
#     fname_movie = glob.glob(fname_slp[:-10]+"*.mp4")[0]
#     print ("movie: ", fname_movie)
    
#     #### FIND ALL ID SWITCHES IN MOVIE ####
#     all_frames = []
    
#     for animal_id in animal_ids:
#     #for animal_id in [2]:
#         frame_ids = detect_id_switch_no_plot(fname_movie,
#                                              track.tracks, 
#                                              animal_id)
#         if len(frame_ids)>0:
#             all_frames.append(frame_ids)
    
#     #
#     all_frames = np.vstack(all_frames)
#     print ("all frames: ", all_frames.shape)
    
#     # load current vid
#     original_vid = cv2.VideoCapture(fname_movie)
#     width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
#     height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )
    
#     # set frames to new ones
#     ctr=0
#     for frames in all_frames:
#         print ("frame #s: ", frames)
#         if len(frames)==0:
#             continue

#         #
#         start = 0
#         original_vid.set(cv2.CAP_PROP_POS_FRAMES, frames[0]+start)
#         for z in range(2):
            
#             ret, img_out = original_vid.read()
#             #plt.figure()
#             #plt.imshow(img_out)
#             #plt.show()
        
#             # save non-annotated video
#             video_out.write(img_out)
            
#             #
#             feats = track.tracks[frames[0]+start+z]

#             #            
#             img_out = plot_feats(
#                                 feats,
#                                 colors,
#                                 img_out,
#                                 dot_size)
            
            
#             video_out_annotated.write(img_out)
#         #video_out_annotated.write(img_out*0)

#     break
    
# video_out.release()
# video_out_annotated.release()
# print ("DONE...")

In [57]:
#
print ("all frames: ", all_frames.shape)
clrs = ['black','blue','red','magenta','cyan','green']
# load current vid
original_vid = cv2.VideoCapture(fname_movie)
width = original_vid.get(cv2.CAP_PROP_FRAME_WIDTH )
height = original_vid.get(cv2.CAP_PROP_FRAME_HEIGHT )

# set frames to new ones
frames = all_frames[0]
    
print ("frame #s: ", frames)

#
plt.figure()
#
for z in range(2):
    ax=plt.subplot(1,2,z+1)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, 
                     frames[z])

    ret, img_out = original_vid.read()

    # save non-annotated video
    video_out.write(img_out)

    #
    feats = track.tracks[frames[z]]
    for k in range(6):
        plt.scatter(feats[k,:,0],
                    feats[k,:,1],
                   c=clrs[k])

    plt.ylim(700,0)
plt.show()

all frames:  (8, 2)
frame #s:  [15000 15001]


In [10]:
####################################################
########## MAKE PLOTS OF SPINE CENTRES #############
####################################################
plt.figure()
for k in range(0,6,1):
    #
    #ax=plt.subplot(2,3,k+1)
    plt.plot(track.tracks_spine[:,k,0],
             track.tracks_spine[:,k,1],
             label='spine2',
             
            )
    #
    # #print (track.tracks.shape)
    # plt.plot(np.nanmedian(track.tracks[:,k,:,0]+2,axis=1),
    #          np.nanmedian(track.tracks[:,k,:,1]+2,axis=1),
    #          label='raw (median of all body parts)',
    #          alpha=.5
    #         )
   
    if k==1:
        plt.legend()
    plt.xlim(0,1000)
    plt.ylim(0,800)
    #break
plt.show()

In [13]:
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
print ("")
print ("Making movie...")

#
vis = Visualize()
fps = 25
start = 0*fps       # frame to start in seconds
end = 60*fps        # frame to end in seconds
fps = 25

#
fname_video = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed.mp4'
fname_video_extension = '_fixed'  # the extension to be added to the filename

# Note; thre is also a version of this that plots all body parts
# - this function takes in only a single animal location 
shrink = 1 # shrink image factor

fname_track = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed_Day_huddle_multi_track_huddles.npy'
track = np.load(fname_track)
print ("track shape: ", track.shape)
#print (track.tracks_spine.shape)
#
vis.make_video_centroid(track,                # visualize centres locations
                        fname_video,
                        fname_video_extension,
                        start,
                        end,
                        fps,
                        shrink) 

# 
print ("DONE...")

  0%|          | 5/1500 [00:00<00:32, 46.07it/s]


Making movie...
track shape:  (28802, 10, 2)
width, heigh:  1280.0 1024.0
Histories:  (10, 5, 2)


100%|██████████| 1500/1500 [00:28<00:00, 52.92it/s]

DONE...
